# --------------------preprocess--------------------

# Import packages

In [1]:
#python 3.6.13
import pickle
import pandas as pd
import sys
from collections import Counter
from multiprocessing import Pool

import numpy as np
import os
import scipy.signal as sg
import wfdb
from tqdm import tqdm
#install opencv-python==4.5.5.64 first then U can install biosppy
import biosppy

# data

In [2]:
sampling_rate = 250
invalid_labels = ['|', '~', '!', '+', '[', ']', '"', 'x']  # non-beat labels
before = 63
after = 77
tol = 0.05

#write data link
base_dir="C:/Users/ASUS/Desktop/cg working/HRV/羅孟宗老師 已標註波形/羅孟宗老師 已標註波形/"
ECGfile=["完成檔raw data/B257_2016_09_07-14_03_09_ECG.csv",\
         "完成檔raw data/B268_2016_12_13-14_34_26_ECG.csv",\
         "完成檔raw data/B268_2016_12_14-10_12_18_ECG.csv",\
         "完成檔raw data/B357#1_2017_09_12-11_10_13_ECG.csv",\
         "完成檔raw data/C727_2016_08_17-14_35_55_ECG.csv",\
         "完成檔raw data/C727_2016_08_18-14_15_50_ECG.csv",\
         "完成檔raw data/C727_2016_08_19-09_00_24_ECG.csv",\
         "完成檔raw data/C733_2016_08_24-09_23_45_ECG.csv",\
         "完成檔raw data/C733_2016_08_25-15_41_37_ECG.csv",\
         "完成檔raw data/C733_2016_08_26-11_50_31_ECG.csv",\
         "完成檔raw data/C783_2016_10_04-15_38_04_ECG.csv",\
         "完成檔raw data/C790_2016_10_28-09_36_26_ECG.csv",\
         "完成檔raw data/C790_2016_10_28-09_59_36_ECG.csv",\
         "完成檔raw data/C790_2016_10_28-10_54_43_ECG.csv",\
         "完成檔raw data/C790_2016_10_28-14_51_20_ECG.csv",\
         "完成檔raw data/C795_2016_11_02-11_43_05_ECG.csv",\
         "完成檔raw data/C795_2016_11_03-09_28_28_ECG.csv",\
         "完成檔raw data/C795_2016_11_03-14_28_24_ECG.csv",\
         "完成檔raw data/C795_2016_11_04-07_48_32_ECG.csv",\
         "完成檔raw data/C807_2016_11_16-11_46_27_ECG.csv",\
         "完成檔raw data/C807_2016_11_18-07_31_57_ECG.csv",\
         "完成檔raw data/C818_2016_11_29-14_52_43_ECG.csv",\
         "完成檔raw data/C818_2016_11_29-15_08_33_ECG.csv",\
         "完成檔raw data/P119#1_2017_08_30-10_46_36_ECG.csv"]
Markfile=["標註在raw data 上/B257_2016_09_07-14_03_09_True_rr.csv",\
          "標註在raw data 上/B268_2016_12_13-14_34_26_True_rr.csv",\
          "標註在raw data 上/B268_2016_12_14-10_12_18_True_rr.csv",\
          "標註在raw data 上/B357-1_2017_09_12-11_10_13_True_rr.csv",\
          "標註在raw data 上/C727_2016_08_17-14_35_55_True_rr.csv",\
          "標註在raw data 上/C727_2016_08_18-14_15_50_True_rr.csv",\
          "標註在raw data 上/C727_2016_08_19-09_00_24_True_rr.csv",\
          "標註在raw data 上/C733_2016_08_24-09_23_45_True_rr.csv",\
          "標註在raw data 上/C733_2016_08_25-15_41_37_True_rr.csv",\
          "標註在raw data 上/C733_2016_08_26-11_50_31_True_rr.csv",\
          "標註在raw data 上/C783_2016_10_04-15_38_04_True_rr.csv",\
          "標註在raw data 上/C790_2016_10_28-09_36_26_True_rr.csv",\
          "標註在raw data 上/C790_2016_10_28-09_59_36_True_rr.csv",\
          "標註在raw data 上/C790_2016_10_28-10_54_43_True_rr.csv",\
          "標註在raw data 上/C790_2016_10_28-14_51_20_True_rr.csv",\
          "標註在raw data 上/C795_2016_11_02-11_43_05_True_rr.csv",\
          "標註在raw data 上/C795_2016_11_03-09_28_28_True_rr.csv",\
          "標註在raw data 上/C795_2016_11_03-14_28_24_True_rr.csv",\
          "標註在raw data 上/C795_2016_11_04-07_48_32_True_rr.csv",\
          "標註在raw data 上/C807_2016_11_16-11_46_27_True_rr.csv",\
          "標註在raw data 上/C807_2016_11_18-07_31_57_True_rr.csv",\
          "標註在raw data 上/C818_2016_11_29-14_52_43_True_rr.csv",\
          "標註在raw data 上/C818_2016_11_29-15_08_33_True_rr.csv",\
          "標註在raw data 上/P119-1_2017_08_30-10_46_36_True_rr.csv"]

#write save link (after preprocess and we will get training data & testing data)
link="C:/Users/ASUS/Desktop/cg working/HRV/Identify correct wave methods/Deep-Neural-Network-For-Heartbeat-Classification-master/"

# preprocess function

In [3]:
def pre_processing(EcgWaveform,Mark):
    filename = EcgWaveform
    signal = pd.read_csv(filename)["EcgWaveform"]
    signal=np.array(list(signal),dtype="f8")

    baseline = sg.medfilt(sg.medfilt(signal, int(0.2 * sampling_rate) - 1), int(0.6 * sampling_rate) - 1)
    signal = signal - baseline

    lab=pd.read_csv(Mark)
    r_peaks=np.array(list(round(lab["position"])),dtype="int")
    labels=np.array(list(lab["mark"]))
    
    # align r-peaks
    newR = []
    for r_peak in r_peaks:
        r_left = np.maximum(r_peak - int(tol * sampling_rate), 0)
        r_right = np.minimum(r_peak + int(tol * sampling_rate), len(signal))
        newR.append(r_left + np.argmax(signal[r_left:r_right]))
    r_peaks = np.array(newR, dtype="int")

    # r-peaks intervals
    rris = np.array(lab["RR"])

    avg_rri = np.mean(rris)
    x1, x2, y = [], [], []
    for index in tqdm(range(len(r_peaks)), file=sys.stdout):
        if index == 0 or index == len(r_peaks) - 1:
            continue
        beat = signal[r_peaks[index] - before: r_peaks[index] + after]

        pre_rri = rris[index - 1]
        post_rri = rris[index]
        ratio_rri = pre_rri / post_rri
        local_rri = np.mean(rris[np.maximum(index - 10, 0):index])

        if labels[index] in ["N", "L", "R", "e", "j"]:
            label = 0  # N
        elif labels[index] in ["A", "a", "S", "J"]:
            label = 1  # SVEB
        elif labels[index] in ["V", "E"]:
            label = 1  # VEB
        elif labels[index] in ["F"]:
            label = 1  # F
        elif labels[index] in ["/", "f", "Q"]:
            label = 1  # Q
        else:
            continue

        x1.append(beat)
        x2.append([pre_rri - avg_rri, post_rri - avg_rri, ratio_rri, local_rri - avg_rri])
        y.append(label)

    return x1, x2, y

# run

In [4]:
x1_train, x2_train, y_train = [], [], []
x1_test, x2_test, y_test = [], [], []
np.random.seed(1225)
for i in range(len(ECGfile)):
    x1,x2,y=pre_processing(os.path.join(base_dir, ECGfile[i]),os.path.join(base_dir, Markfile[i]))
    a=np.floor(0.8*len(y))
    if (0.8*len(y)-a)>=0.5:
        a+1
    b=len(y)-a
    a=int(a)
    b=int(b)
    rdgroup=np.random.choice([0]*a+[1]*b,len(y),replace=False)

    x1tn,x2tn,ytn,x1tt,x2tt,ytt=[],[],[],[],[],[]
    for i in range(len(y)):
        if rdgroup[i]==1:
            x1tt.append(x1[i])
            x2tt.append(x2[i])
            ytt.append(y[i])
        if rdgroup[i]==0:
            x1tn.append(x1[i])
            x2tn.append(x2[i])
            ytn.append(y[i])
    
    x1_train.append(x1tn)
    x2_train.append(x2tn)
    y_train.append(ytn)
    x1_test.append(x1tt)
    x2_test.append(x2tt)
    y_test.append(ytt)

x1_train = np.concatenate(x1_train, axis=0)
x2_train = np.concatenate(x2_train, axis=0)
y_train = np.concatenate(y_train, axis=0)
x1_test = np.concatenate(x1_test, axis=0)
x2_test = np.concatenate(x2_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

with open(os.path.join(link, "CNNMPFL_preprocess.pkl"), "wb") as f:
        pickle.dump((
            (x1_train, x2_train, y_train),
            (x1_test, x2_test, y_test)
        ), f, protocol=4)

print("train labels:", Counter(y_train))
print("test labels:", Counter(y_test))


100%|██████████| 148/148 [00:00<?, ?it/s]
train labels: Counter({0: 49120, 1: 339})
test labels: Counter({0: 12304, 1: 74})


# --------------------training--------------------

# Import packages

In [5]:
from datetime import datetime

import keras
import numpy as np
import os
from keras import backend as K
from keras.callbacks import LearningRateScheduler
from keras.callbacks import TensorBoard
from keras.layers import Activation, BatchNormalization, Concatenate, Conv1D, Dense, Flatten, Input, MaxPooling1D
from keras.models import Model
from keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler
import random
import tensorflow as tf

In [6]:
#num_classes
p=2
#shape
shp=140
#wtraining data & testing data link
link2=link

#ita for focal loss function
ita=0

# parameters in paper ita=2, epoch=50, batch size=512, CNN_kernel_size=(11,5,3), CNN_kernel_num=(16,32,64), CNN_strides=(3,1,1), 
# max_pool=(3,2), final_relu=64
# parameters of mine ita=, epoch=50, batch size=512, CNN_kernel_size=(6,11,4), CNN_kernel_num=(32,32,64), CNN_strides=(2,1,1), 
# max_pool=(3,2), final_relu=16

# model function

In [7]:
def f1(y_true, y_pred):
    y_pred = K.one_hot(K.argmax(y_pred, axis=-1), num_classes=p)
    y_true = K.cast(y_true, y_pred.dtype)

    tp = K.sum(y_true * y_pred, axis=0)
    tn = K.sum((1 - y_true) * (1 - y_pred), axis=0)
    fp = K.sum((1 - y_true) * y_pred, axis=0)
    fn = K.sum(y_true * (1 - y_pred), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    return K.mean(2 * precision * recall / (precision + recall + K.epsilon()))


def categorical_focal_loss(gamma=ita):
    """
        Categorical form of focal loss.
            FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
        References:
            https://arxiv.org/pdf/1708.02002.pdf
        Usage:
            model.compile(loss=categorical_focal_loss(gamma=2), optimizer="adam", metrics=["accuracy"])
            model.fit(class_weight={0:alpha0, 1:alpha1, ...}, ...)
        Notes:
           1. The alpha variable is the class_weight of keras.fit, so in implementation of the focal loss function
           we needn't define this variable.
           2. (important!!!) The output of the loss is the loss value of each training sample, not the total or average
            loss of each batch.
    """

    def focal_loss(y_true, y_pred):
        y_true = K.cast(y_true, y_pred.dtype)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        return K.sum(K.sum(-y_true * K.pow(1 - y_pred, gamma) * K.log(y_pred), axis=-1))

    return focal_loss


def create_model(l=0.0):
    inputs1 = Input(shape=(shp, 1))
    x1 = inputs1

    x1 = Conv1D(16, kernel_size=8, strides=2, kernel_initializer="he_normal", kernel_regularizer=l2(l),
                bias_regularizer=l2(l))(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation("relu")(x1)

    x1 = MaxPooling1D(3, strides=2)(x1)

    x1 = Conv1D(32, kernel_size=5, kernel_initializer="he_normal", kernel_regularizer=l2(l),
                bias_regularizer=l2(l))(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation("relu")(x1)

    x1 = MaxPooling1D(3, strides=2)(x1)

    x1 = Conv1D(64, kernel_size=3, kernel_initializer="he_normal", kernel_regularizer=l2(l),
                bias_regularizer=l2(l))(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation("relu")(x1)

    x1 = MaxPooling1D(3, strides=2)(x1)

    x1 = Flatten()(x1)

    inputs2 = Input(shape=(4,))
    x2 = inputs2

    x = Concatenate()([x1, x2])

    x = Dense(64, kernel_initializer="he_normal", kernel_regularizer=l2(l), bias_regularizer=l2(l),
              activation="relu")(x)

    outputs = Dense(p, activation="softmax")(x)

    model = Model(inputs=(inputs1, inputs2), outputs=outputs)

    return model


class MyGenerator(keras.utils.data_utils.Sequence):

    def __init__(self, x1, x2, y, batch_size):
        self.x1 = x1
        self.x2 = x2
        self.y = y
        self.batch_size = batch_size
        self.indices = np.arange(len(self.x1))
        self.on_epoch_end()

    def __len__(self):
        return len(self.x1) // self.batch_size + 1

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

    def __getitem__(self, item):
        x1_batch = self.x1[self.indices[item * self.batch_size:(item + 1) * self.batch_size]]
        x2_batch = self.x2[self.indices[item * self.batch_size:(item + 1) * self.batch_size]]
        y_batch = self.y[self.indices[item * self.batch_size:(item + 1) * self.batch_size]]
        return (x1_batch, x2_batch), y_batch


def load_data(filename):
    #load pkl file
    import pickle

    with open(filename, "rb") as f:
        (x1_train, x2_train, y_train), (x1_test, x2_test, y_test) = pickle.load(f)

    return (x1_train, x2_train, y_train), (x1_test, x2_test, y_test)


def main(filename):
    epochs = 50
    batch_size = 512

    # loading data
    (x1_train, x2_train, y_train), (x1_test, x2_test, y_test) = load_data(filename)

    x1_train = np.expand_dims(x1_train, axis=-1)
    x1_test = np.expand_dims(x1_test, axis=-1)

    scaler = RobustScaler()
    x2_train = scaler.fit_transform(x2_train)
    x2_test = scaler.transform(x2_test)

    y_train = keras.utils.np_utils.to_categorical(y_train, num_classes=p)
    y_test = keras.utils.np_utils.to_categorical(y_test, num_classes=p)

    train_generator = MyGenerator(x1_train, x2_train, y_train, batch_size)
    test_generator = MyGenerator(x1_test, x2_test, y_test, batch_size)

    model = create_model(l=1e-3)

    # callbacks
    log_dir = os.path.join("./logs", datetime.now().strftime("%H-%M-%S"))
    tb_cb = TensorBoard(log_dir=log_dir)

    def schedule(epoch, lr):
        if (epoch + 1) % 10 == 0:
            lr *= 0.1
        return lr

    lr_scheduler = LearningRateScheduler(schedule=schedule, verbose=0)

    # training
    model.compile(loss=categorical_focal_loss(gamma=ita), optimizer="adam", metrics=["acc", f1])
    model.fit(train_generator, epochs=epochs, verbose=1, callbacks=[tb_cb, lr_scheduler],
              validation_data=test_generator)

    model.save(os.path.join("./models", "model_crossentropy.h5"))
    #model_focalloss(2)

    #result(train)
    y_true = np.argmax(y_train, axis=-1)
    y_pred = np.argmax(model.predict([x1_train, x2_train], batch_size=batch_size, verbose=1), axis=-1)

    print("training data")
    print("row:true, column:pred")
    print(pd.DataFrame(confusion_matrix(y_true, y_pred)))
    print("classification_report")
    print(classification_report(y_true, y_pred, digits=4))

    #result(test)
    y_true = np.argmax(y_test, axis=-1)
    y_pred = np.argmax(model.predict([x1_test, x2_test], batch_size=batch_size, verbose=1), axis=-1)

    print("testing data")
    print("row:true, column:pred")
    print(pd.DataFrame(confusion_matrix(y_true, y_pred)))
    print("classification_report")
    print(classification_report(y_true, y_pred, digits=4))



# training

In [8]:
(x1_train, x2_train, y_train), (x1_test, x2_test, y_test)=load_data(os.path.join(link2, "CNNMPFL_preprocess.pkl"))


In [9]:
x1_train = np.expand_dims(x1_train, axis=-1)
x1_test = np.expand_dims(x1_test, axis=-1)

scaler = RobustScaler()
x2_train = scaler.fit_transform(x2_train)
x2_test = scaler.transform(x2_test)

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes=p)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes=p)

print("train labels:", np.sum(y_train, axis=0))
print("test labels:", np.sum(y_test, axis=0))

train labels: [49120.   339.]
test labels: [12304.    74.]


In [10]:
model = create_model(l=1e-3)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 140, 1)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 67, 16)       144         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 67, 16)       64          conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 67, 16)       0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [11]:
def seed_every(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
seed_every(61926)
CNNMPFL_modeling=main(os.path.join(link2, "CNNMPFL_preprocess.pkl"))


Epoch 1/50
97/97 [==============================] - 5s 37ms/step - loss: 60.3062 - acc: 0.9781 - f1: 0.5404 - val_loss: 28.3763 - val_acc: 0.9885 - val_f1: 0.6501
Epoch 2/50
97/97 [==============================] - 3s 35ms/step - loss: 15.0753 - acc: 0.9939 - f1: 0.6940 - val_loss: 13.2407 - val_acc: 0.9943 - val_f1: 0.7169
Epoch 3/50
97/97 [==============================] - 3s 36ms/step - loss: 11.0149 - acc: 0.9948 - f1: 0.7392 - val_loss: 9.7113 - val_acc: 0.9955 - val_f1: 0.7409
Epoch 4/50
97/97 [==============================] - 3s 36ms/step - loss: 9.3701 - acc: 0.9955 - f1: 0.7711 - val_loss: 15.5332 - val_acc: 0.9943 - val_f1: 0.7212
Epoch 5/50
97/97 [==============================] - 3s 35ms/step - loss: 9.9496 - acc: 0.9956 - f1: 0.7951 - val_loss: 8.6415 - val_acc: 0.9956 - val_f1: 0.7206
Epoch 6/50
97/97 [==============================] - 3s 35ms/step - loss: 7.8747 - acc: 0.9959 - f1: 0.8209 - val_loss: 8.3620 - val_acc: 0.9956 - val_f1: 0.7518
Epoch 7/50
97/97 [==========